In [1]:
import os
import sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from basefactor import fetcher
from basicfunc import *
from localsimulator import *
from dataclean import *

c:\Users\hazc\anaconda3\Lib\site-packages\iFinDPy.pth


# 检查一个完整的模拟流程

## 1. 对比一下数据准确性

In [2]:
target_list = ["ASHARE", "CAP", "CLOSE", "DAYS", "HIGH", "ISZT",\
               "LOW", "OPEN", "PE", "STOCKS", "TVR5", "VOL", "VWAP",\
               "VWAPRET", "WIND01"]
start_date = "2021-01-01"
end_date = "2022-12-31"
df_1 = fetcher.get_market_data('jydb.dstock', start_date, end_date, "CLOSE", "PCT_CHG",\
                               "MAXUPORDOWN", "VAL_FLOATMV", "is_trade", "FLOAT_A_SHARES", "EV")
df_2 = fetcher.get_tradedate_data('jydb.dailydata', start_date, end_date, "freefloats",\
                                  "firstindustrycode", "ifsuspend", "changepct", "negotiablemv",\
                                  "closeprice", "totalmv")
df_1 = df_1[~df_1['wind_code'].str.endswith('.BJ')]

def process_data(df):
    df_new = df.set_index(['tradedate', 'wind_code'])
    return df_new.sort_index()

df_1_new = process_data(df_1.copy())
df_2_new = process_data(df_2.copy())


In [11]:
df_1_new.xs('600145.SH', level=1)['2022-04-25':'2022-05-20']

,CLOSE,PCT_CHG,MAXUPORDOWN,VAL_FLOATMV,is_trade,FLOAT_A_SHARES,EV
tradedate,,,,,,,
2022-04-25,0.34,0.0,0.0,3.004642e+08,0.0,1.491100e+09,506974129.2
2022-04-26,0.34,0.0,0.0,3.004642e+08,0.0,1.491100e+09,506974129.2


In [12]:
df_2_new.xs('600145.SH', level=1)['2022-04-25':'2022-05-20']

,freefloats,firstindustrycode,ifsuspend,changepct,negotiablemv,closeprice,totalmv
tradedate,,,,,,,
2022-04-25,883718284.0,31.0,1.0,0.0,506974129.2,0.34,506974129.2
2022-04-26,883718284.0,31.0,1.0,0.0,506974129.2,0.34,506974129.2
2022-04-27,883718284.0,31.0,1.0,0.0,506974129.2,0.34,506974129.2
